# Squashed Entanglement
$$
 \newcommand{\ul}[1]{\underline{#1}}
 \newcommand{\rvalp}[0]{{\ul{\alpha}}}
\newcommand{\alp}[0]{{\alpha}}
 \newcommand{\rvx}[0]{{\ul{x}}}
\newcommand{\rvy}[0]{{\ul{y}}}
$$

The purpose of this notebook is to show how to use entanglish to
calculate the squashed entanglement of a mixed state (either pure or not pure).

Consider a bipartite system
consisting of two parts labelled
by the random variables $\rvx$ and $\rvy$,
and
described by a density matrix $\rho_{\rvx, \rvy}$.
The squashed entanglement of  such a system
is defined as

$$
E_{\rvx, \rvy}(\rho_{\rvx, \rvy}) =
\frac{1}{2}
\min S(\rvx : \rvy|\rvalp)
\;.
$$
The min()---or infimum()
if one wishes to be more mathematically
precise--is
over all density matrices $\rho_{\rvx, \rvy,\rvalp}$
such that ${\rm tr}_\rvalp \; \rho_{\rvx, \rvy,\rvalp}=
\rho_{\rvx, \rvy}$ with $\rho_{\rvx, \rvy}$ held fixed.
If $\rho_{\rvx, \rvy}$ is a pure state, then
$E_{\rvx, \rvy} = S(\rvx) = S(\rvy)$.
Entanglish-Original-Ref discusses other interesting
properties of squashed entanglement

Entanglish-Original-Ref also describes the algo
used by Entanglish to calculate squashed entanglement. The
algorithm is recursive. 
The number of recursive steps 
can be chosen by the user and is called num_ab_steps (ab stands
for Arimoto-Blahut).
Another parameter of the algorithm is num_hidden_states, which is the number of possible 
$\rvalp$ values.

 
**Entanglish-Original-Ref**
* "A New  Algorithm for Calculating
Squashed Entanglement and a Python Implementation Thereof", by R.R.Tucci

First change your working directory to the entanglish directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/rrtucci/PycharmProjects/Entanglish/entanglish/jupyter_notebooks
/home/rrtucci/PycharmProjects/Entanglish


In [2]:
from entanglish.SymNupState import *
from entanglish.SquashedEnt import *

## pure states (symmetrized n-up states)
Next we construct a symmetrized n-up pure state.
Then we compare the Arimoto-Blahut algo entanglement value to the known 
entanglement value, for various possible
bi-partitions of the set of row axes. As expected, they are equal.

recursion_init is a str that specifies how to
initiate the recursion for Kxy_a. There are currently 2 options for recursion_init, 'eigen' and 'eigen+'. 'eigen+' uses Dxy.num_rows^2 degrees of freedom, which is sufficient but 
may not be necessary. 'eigen' uses only Dxy.num_rows degrees of freedom.
With both 'eigen' and 'eigen+', the recursion converges instantly for pure states. 

In [3]:
num_qbits = 4
num_up = 1
dm1 = DenMat(1 << num_qbits, tuple([2]*num_qbits))
st = SymNupState(num_up, num_qbits)
st_vec = st.get_st_vec()
dm1.set_arr_from_st_vec(st_vec)

recursion_init = 'eigen+'
num_ab_steps = 5
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
ecase = SquashedEnt(dm1, num_ab_steps,
    recursion_init=recursion_init, verbose=True)
print('entang_023: algo value, known value\n',
      ecase.get_entang({0, 2, 3}),
      st.get_known_entang(3))
print('entang_02: algo value, known value\n',
      ecase.get_entang({0, 2}),
      st.get_known_entang(2))
print('entang_1: algo value, known value\n',
      ecase.get_entang({1}),
      st.get_known_entang(1))

recursion_init= eigen+
num_ab_steps= 5

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.562335 , err= 0.000000
--ab step= 1 , entang= 0.562335 , err= 0.000000
--ab step= 2 , entang= 0.562335 , err= 0.000000
--ab step= 3 , entang= 0.562335 , err= 0.000000
--ab step= 4 , entang= 0.562335 , err= 0.000000
entang_023: algo value, known value
 0.5623351403283497 0.5623351446188083

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.693147 , err= 0.000000
--ab step= 1 , entang= 0.693147 , err= 0.000000
--ab step= 2 , entang= 0.693147 , err= 0.000000
--ab step= 3 , entang= 0.693147 , err= 0.000000
--ab step= 4 , entang= 0.693147 , err= 0.000000
entang_02: algo value, known value
 0.6931471760470338 0.6931471805599453

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 2.7194799110210365e-16
--ab step= 0 , entang= 0.562335 , err= 0.000000
--ab step= 1 , entang= 0.562335 , err= 0.000000
--ab ste

## random density matrices
Next we consider 2 random density matrices (actually,
only their eigenvectors are random. Their eigenvalues are specified by the user.)
For each of those 2 density matrices, we calculate the Arimoto-Blahut algo entanglement value, for various possible
bi-partitions of the set of row axes.

In [4]:
np.random.seed(123)
dm = DenMat(8, (2, 2, 2))
evas_of_dm_list = [
    np.array([.07, .03, .25, .15, .3, .1, .06, .04])
    , np.array([.05, .05, .2, .2, .3, .1, .06, .04])
    ]

recursion_init = 'eigen+'
num_ab_steps = 100
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
for evas_of_dm in evas_of_dm_list:
    evas_of_dm /= np.sum(evas_of_dm)
    print('***************new dm')
    print('evas_of_dm\n', evas_of_dm)
    dm.set_arr_to_rand_den_mat(evas_of_dm)
    ecase = SquashedEnt(dm, num_ab_steps,
        recursion_init=recursion_init, verbose=True)
    print('ent_02_1=', ecase.get_entang({0, 2}))

recursion_init= eigen+
num_ab_steps= 100
***************new dm
evas_of_dm
 [0.07 0.03 0.25 0.15 0.3  0.1  0.06 0.04]

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 4.880055036277153e-16
--ab step= 0 , entang= 0.446881 , err= 26.212601
--ab step= 1 , entang= 0.070777 , err= 16.767193
--ab step= 2 , entang= 0.059759 , err= 3.805233
--ab step= 3 , entang= 0.065865 , err= 1.799566
--ab step= 4 , entang= 0.066359 , err= 1.067930
--ab step= 5 , entang= 0.064568 , err= 0.810175
--ab step= 6 , entang= 0.061791 , err= 0.721966
--ab step= 7 , entang= 0.058829 , err= 0.677880
--ab step= 8 , entang= 0.055934 , err= 0.636697
--ab step= 9 , entang= 0.053073 , err= 0.593049
--ab step= 10 , entang= 0.050332 , err= 0.551687
--ab step= 11 , entang= 0.047747 , err= 0.516593
--ab step= 12 , entang= 0.045331 , err= 0.488230
--ab step= 13 , entang= 0.043090 , err= 0.465334
--ab step= 14 , entang= 0.041023 , err= 0.446307
--ab step= 15 , entang= 0.039133 , err= 0.429767
--ab step= 16 , entang= 0.037422

--ab step= 61 , entang= 0.003721 , err= 0.199322
--ab step= 62 , entang= 0.003668 , err= 0.200231
--ab step= 63 , entang= 0.003619 , err= 0.200451
--ab step= 64 , entang= 0.003573 , err= 0.199992
--ab step= 65 , entang= 0.003532 , err= 0.198879
--ab step= 66 , entang= 0.003496 , err= 0.197152
--ab step= 67 , entang= 0.003465 , err= 0.194854
--ab step= 68 , entang= 0.003439 , err= 0.192037
--ab step= 69 , entang= 0.003419 , err= 0.188748
--ab step= 70 , entang= 0.003404 , err= 0.185035
--ab step= 71 , entang= 0.003395 , err= 0.180961
--ab step= 72 , entang= 0.003390 , err= 0.176534
--ab step= 73 , entang= 0.003391 , err= 0.171775
--ab step= 74 , entang= 0.003396 , err= 0.166730
--ab step= 75 , entang= 0.003405 , err= 0.161471
--ab step= 76 , entang= 0.003418 , err= 0.156089
--ab step= 77 , entang= 0.003435 , err= 0.150653
--ab step= 78 , entang= 0.003454 , err= 0.145115
--ab step= 79 , entang= 0.003475 , err= 0.139603
--ab step= 80 , entang= 0.003499 , err= 0.134201
--ab step= 81 , enta